# Final Project
Clay Kindiger and Haoyang Pang

## Load Libraries

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Load Original Data

In [6]:
players_team = pd.read_csv('full_player_stats.csv')
advanced_stats = pd.read_csv('data/advanced_players.csv')
mvp_voting = pd.read_csv('data/mvp_voting_shares.csv')
dpoy_voting = pd.read_csv('data/dpoy_voting.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 20913: invalid start byte

In [ ]:
players_team.head()

In [ ]:
advanced_stats.head()

In [ ]:
mvp_voting.head()

In [ ]:
dpoy_voting.head()

## Clean Data

In [ ]:
players_team.drop(columns = ['mvp','dpoy'],inplace = True)

In [ ]:
season = []
start = 2023
for idx,row in dpoy_voting.iterrows():
    if row['Rank'] == '1':
        start -= 1
    
    season.append(start)
dpoy_voting['Season'] = season

In [ ]:
dpoy_voting = dpoy_voting[['Season', 'Player',  'Pts Won', 'Pts Max', 'Share']]
dpoy_voting.head()

In [ ]:
advanced_stats = advanced_stats[advanced_stats['Player'] != 'Player']

In [ ]:
players_teams = {}
for ply,group in advanced_stats.groupby(['Player','Season']):
    if group.shape[0] > 1:
        tm = list(group.Tm)[-1]
        if 28 in group.index:
            print(group.Tm)
        players_teams[list(group.index)[0]] = tm
        if tm == 'TOT':
            print(ply)

In [ ]:
teams_list = []
for idx,row in advanced_stats.iterrows():
    if idx in players_teams.keys():
        tm = players_teams[idx]
        teams_list.append(tm)
    else:
        tm = row['Tm']
        if idx == 3613:
            tm = 'UTA'
        elif idx == 21:
            tm = 'MIA'
        elif idx == 4189:
            tm = 'WAS'
        elif idx == 3037:
            tm = 'BOS'
        elif idx == 11099:
            tm = 'CHI'
        elif idx == 6128:
            tm = 'NJN'
        teams_list.append(tm)
advanced_stats['Tm'] = teams_list

In [ ]:
advanced_stats.drop_duplicates(['Player','Season'],keep = 'first',inplace = True)

In [ ]:
players_team.shape

In [ ]:
advanced_stats.shape

In [ ]:
advanced_stats.columns

In [ ]:
players_team.columns

In [ ]:
advanced_stats = advanced_stats[['Season', 'Player', 'Tm', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]

In [ ]:
full_players = players_team.merge(advanced_stats, on = ['Player','Tm','Season'],how = 'inner')
full_players.shape

In [ ]:
full_players.head()

In [ ]:
full_players_mvp = full_players.merge(mvp_voting,on = ['Season','Player'],how = 'left')
full_players_mvp.shape

In [ ]:
full_players_mvp

In [ ]:
full_players_mvp[~full_players_mvp['Share'].isnull()]

In [ ]:
full_players_awards = full_players_mvp.merge(dpoy_voting,on = ['Season','Player'],how = 'left',suffixes = ('_mvp','_dpoy'))
full_players_awards.shape

In [ ]:
full_players_awards.fillna(0,inplace = True)

In [ ]:
full_players_awards.to_csv('full_players_dataset.csv',index = False)

In [ ]:
full_dataset = pd.read_csv('full_players_dataset.csv')
full_dataset.head()

In [ ]:
full_dataset.columns

In [ ]:
full_dataset['received_mvp_votes'] = [1 if row['Share_mvp']!=0 else 0 for idx,row in full_dataset.iterrows()]
full_dataset['received_dpoy_votes'] = [1 if row['Share_dpoy']!=0 else 0 for idx,row in full_dataset.iterrows()]

In [ ]:
cols_to_normalize = ['G','GS','W','L','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']

In [ ]:
normalized = full_dataset.groupby('Season')[cols_to_normalize].transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
full_normalized = full_dataset.merge(normalized,right_index = True,left_index=True,suffixes = ('','_normalized'))

In [ ]:
full_normalized.head()

In [ ]:
full_normalized.drop(cols_to_normalize,axis = 1,inplace=True)

In [ ]:
full_normalized.head()

In [ ]:
full_normalized.to_csv("normalized_data.csv",index=False)

## Load Cleaned Data

In [7]:
full_dataset = pd.read_csv('normalized_data.csv')
full_dataset.head()

,Season,Player,Pos,Age,Tm,MP,Year,Team,W/L%,GB,...,3PAr_normalized,FTr_normalized,ORB%_normalized,DRB%_normalized,TRB%_normalized,AST%_normalized,STL%_normalized,BLK%_normalized,TOV%_normalized,USG%_normalized
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,530.0,1998,SAC,0.329,34.0,...,-0.223145,-1.479163,-1.061591,-1.082926,-1.256266,0.678781,-0.270151,-0.824493,-1.271023,1.028959
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,959.0,1998,SAC,0.329,34.0,...,-0.577654,0.059612,-0.333466,-0.775932,-0.686891,-0.475951,0.010661,-0.352740,-0.260471,0.499256
2,1998,Terry Dehere,PG,26.0,SAC,1410.0,1998,SAC,0.329,34.0,...,0.817135,-0.627813,-0.984946,-1.050611,-1.195261,0.967465,0.010661,-0.765524,0.118486,-0.118731
3,1998,Lawrence Funderburke,PF,27.0,SAC,1094.0,1998,SAC,0.329,34.0,...,-0.781061,0.403325,0.241370,0.419729,0.370520,-0.358339,-0.925378,-0.352740,-0.513109,0.358002
4,1998,Derek Grimm,SF,23.0,SAC,34.0,1998,SAC,0.329,34.0,...,4.094891,-1.034981,-1.291525,-0.000368,-0.727560,-1.427536,2.537968,0.295920,0.188663,0.728794


## Feature Selection

In [8]:
full_dataset.columns

Index(['Season', 'Player', 'Pos', 'Age', 'Tm', 'MP', 'Year', 'Team', 'W/L%',
       'GB', 'PS/G', 'PA/G', 'SRS', 'Playoffs', 'Losing_season', 'OWS', 'DWS',
       'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Pts Won_mvp',
       'Pts Max_mvp', 'Share_mvp', 'Pts Won_dpoy', 'Pts Max_dpoy',
       'Share_dpoy', 'received_mvp_votes', 'received_dpoy_votes',
       'G_normalized', 'GS_normalized', 'W_normalized', 'L_normalized',
       'FG_normalized', 'FGA_normalized', 'FG%_normalized', '3P_normalized',
       '3PA_normalized', '3P%_normalized', '2P_normalized', '2PA_normalized',
       '2P%_normalized', 'eFG%_normalized', 'FT_normalized', 'FTA_normalized',
       'FT%_normalized', 'ORB_normalized', 'DRB_normalized', 'TRB_normalized',
       'AST_normalized', 'STL_normalized', 'BLK_normalized', 'TOV_normalized',
       'PF_normalized', 'PTS_normalized', 'PER_normalized', 'TS%_normalized',
       '3PAr_normalized', 'FTr_normalized', 'ORB%_normalized',
       'DRB%_normalized', 'TRB%_normal

## Create Models

### Random Forest

In [9]:
mvp_dataset = full_dataset[['Playoffs', 'OWS', 'DWS',
       'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'received_mvp_votes',
       'G_normalized', 'GS_normalized', 'W_normalized', 'L_normalized',
       'FG_normalized', 'FGA_normalized', 'FG%_normalized', '3P_normalized',
       '3PA_normalized', '3P%_normalized', '2P_normalized', '2PA_normalized',
       '2P%_normalized', 'eFG%_normalized', 'FT_normalized', 'FTA_normalized',
       'FT%_normalized', 'ORB_normalized', 'DRB_normalized', 'TRB_normalized',
       'AST_normalized', 'STL_normalized', 'BLK_normalized', 'TOV_normalized',
       'PF_normalized', 'PTS_normalized', 'PER_normalized', 'TS%_normalized',
       '3PAr_normalized', 'FTr_normalized', 'ORB%_normalized',
       'DRB%_normalized', 'TRB%_normalized', 'AST%_normalized',
       'STL%_normalized', 'BLK%_normalized', 'TOV%_normalized',
       'USG%_normalized']]

In [10]:
X = np.array(mvp_dataset.drop(['received_mvp_votes'],axis=1))
y = np.array(mvp_dataset['received_mvp_votes'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
clf = RandomForestClassifier(30, oob_score = True).fit(X_train,y_train)

print(f'The training accuracy: {clf.score(X,y)}')
print(f'The out of bag classification accuracy: {clf.oob_score_}')

The training accuracy: 0.9986509274873524
The out of bag classification accuracy: 0.9831365935919055


In [12]:
clf.score(X_test,y_test)

0.9890387858347386

### validate the result with different matrix (new)

In [25]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming you have already trained your classifier 'clf' and have your test sets 'X_test' and 'y_test'

# Predictions for the test set
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:, 1]  # probabilities for ROC-AUC

# Classification Metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("ROC-AUC Score:")
print(roc_auc_score(y_test, y_pred_proba))

# Regression Metrics for the classification predictions
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", mean_squared_error(y_test, y_pred, squared=False))
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1159
           1       0.89      0.59      0.71        27

    accuracy                           0.99      1186
   macro avg       0.94      0.80      0.85      1186
weighted avg       0.99      0.99      0.99      1186

Confusion Matrix:
[[1157    2]
 [  11   16]]
ROC-AUC Score:
0.9717189147732719
Mean Squared Error (MSE): 0.010961214165261383
Root Mean Squared Error (RMSE): 0.10469581732457789
Mean Absolute Error (MAE): 0.010961214165261383
R² Score: 0.5073019525133418


### Apply Regression (New)

In [33]:
from sklearn.ensemble import RandomForestRegressor

#create the new variable for the regression
y_regr = np.array(full_dataset['Share_mvp'])

# need to filter out the training data for players predicted not to receive votes
# predict on the entire dataset
predicted_votes = clf.predict(X)

#filter X and y for regression based on the prediction
X_regr = X[predicted_votes > 0]
y_regr = y_regr[predicted_votes > 0]

X_train_regr, X_test_regr, y_train_regr, y_test_regr = train_test_split(X_regr, y_regr, test_size=0.1, random_state=42)

regr = RandomForestRegressor(n_estimators=40, oob_score=True)
regr.fit(X_train_regr, y_train_regr)

regression_score = regr.score(X_test_regr, y_test_regr)

In [23]:
print(predicted_votes)
candidate=np.where(predicted_votes==1)
print(candidate)

[0 0 0 ... 0 0 0]
(array([    9,   104,   128,   132,   139,   151,   159,   176,   203,
         250,   278,   324,   353,   365,   374,   420,   450,   495,
         509,   537,   556,   584,   587,   601,   612,   626,   650,
         666,   706,   722,   741,   775,   810,   820,   866,   876,
         923,   979,  1048,  1067,  1086,  1112,  1115,  1157,  1181,
        1189,  1205,  1239,  1263,  1273,  1294,  1311,  1356,  1377,
        1395,  1403,  1417,  1434,  1476,  1485,  1529,  1559,  1568,
        1591,  1629,  1667,  1713,  1724,  1734,  1771,  1773,  1800,
        1818,  1820,  1908,  1965,  1984,  2007,  2042,  2057,  2081,
        2090,  2093,  2097,  2110,  2155,  2164,  2209,  2210,  2390,
        2409,  2426,  2454,  2462,  2500,  2558,  2566,  2575,  2613,
        2678,  2703,  2761,  2776,  2796,  2851,  2885,  2898,  2911,
        2933,  2995,  3003,  3024,  3026,  3046,  3063,  3102,  3145,
        3187,  3189,  3196,  3208,  3248,  3306,  3320,  3322,  3327,
 

In [34]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predictions for regression
y_pred_regr = regr.predict(X_test_regr)

# Regression metrics
mse_regr = mean_squared_error(y_test_regr, y_pred_regr)
mae_regr = mean_absolute_error(y_test_regr, y_pred_regr)
r2_regr = r2_score(y_test_regr, y_pred_regr)

print("Regression score:", regression_score)
print("Mean Squared Error (MSE) for Regression:", mse_regr)
print("Mean Absolute Error (MAE) for Regression:", mae_regr)
print("R² Score for Regression:", r2_regr)

Regression score: 0.3491705737284555
Mean Squared Error (MSE) for Regression: 0.06054594390625001
Mean Absolute Error (MAE) for Regression: 0.16820763888888887
R² Score for Regression: 0.3491705737284555


In [35]:
for i in range(20,100,5):
    mvp_regr = RandomForestRegressor(i, oob_score = True).fit(X_train_regr,y_train_regr)
    print(f'Test Accuracy with {i} trees = {mvp_regr.score(X_test_regr,y_test_regr)}')

Test Accuracy with 20 trees = 0.386132955285839
Test Accuracy with 25 trees = 0.3213834840297658
Test Accuracy with 30 trees = 0.34796864518011894
Test Accuracy with 35 trees = 0.3112553250230117
Test Accuracy with 40 trees = 0.40470649052272856
Test Accuracy with 45 trees = 0.43756297231267194
Test Accuracy with 50 trees = 0.27559501975072453
Test Accuracy with 55 trees = 0.3210259263223928
Test Accuracy with 60 trees = 0.36557751913522873
Test Accuracy with 65 trees = 0.37126440979431674
Test Accuracy with 70 trees = 0.3235662544887451
Test Accuracy with 75 trees = 0.390050559489053
Test Accuracy with 80 trees = 0.3412762395702812
Test Accuracy with 85 trees = 0.3558918326998852
Test Accuracy with 90 trees = 0.3378272742495694
Test Accuracy with 95 trees = 0.4187323568881246


### Decision Tree

NameError: name 'X' is not defined

## Evaluate Models